In [15]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile

import os
import pickle
import random
import operator

import math

In [16]:
# function to get the distance between feature vectors and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):                               
        neighbors.append(distances[x][0])
#     print(distances)
#     print(len(neighbors))
    return neighbors


In [17]:
# identify the class of the instance
def nearestClass(neighbors):
    classVote = {}                     

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)      

    return sorter[0][0]                       

In [18]:
# function to evaluate the model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:           
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [19]:
# directory that holds the wav files
directory = "./Data/genres_original/"         

# binary file where we will collect all the features extracted using mfcc (Mel Frequency Cepstral Coefficients)
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):            
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):           
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)          
            covariance = np.cov(np.matrix.transpose(mfcc_feat))                
            mean_matrix = mfcc_feat.mean(0)                                              
            feature = (mean_matrix, covariance, i)                          
            pickle.dump(feature, f)                                        
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported.  in folder:  jazz  filename:  jazz.00054.wav


In [24]:
# Split the dataset into training and testing sets respectively
dataset = []                       

def loadDataset(filename, split, trSet, teSet):                        
    with open('my.dat', 'rb') as f:                  
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

In [21]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [22]:
# making predictions using KNN
leng = len(testSet)
predictions = []     
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.7024793388429752


In [10]:
# testing the code with external samples
# URL: https://uweb.engr.arizona.edu/~429rns/audiofiles/audiofiles.html

# test_dir = "./Test/"
# # test_file = test_dir + "test.wav"
# test_file = test_dir + "test5.wav"
# # test_file = test_dir + "test4.wav"

In [11]:
# (rate, sig) = wav.read(test_file)
# mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
# covariance = np.cov(np.matrix.transpose(mfcc_feat))
# mean_matrix = mfcc_feat.mean(0)
# feature = (mean_matrix, covariance, i)

In [12]:
# from collections import defaultdict
# results = defaultdict(int)

# directory = "./Data/genres_original/"

# i = 1
# for folder in os.listdir(directory):
#     results[i] = folder
#     i += 1

In [13]:
# pred = nearestClass(getNeighbors(dataset, feature, 5))
# print(results[pred])

In [23]:
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from collections import defaultdict

class App:
    def __init__(self, root, i):
        self.root = root                     
        self.file_name = ""
        self.i = i
        self.root.geometry("700x500")
        self.root.title("Music Genre Classification")                  
        lbl = Label(self.root, text="Music Genre Classification", pady=10, bg="#60c6eb", font=("goudy old style", 24, "bold")).place(x=0, y=5, relwidth=1)
        btn = Button(self.root, text="Upload File", bg="#95b8ba", font=("goudy old style", 16, "bold"), command=self.upload).place(x=200, y=120)
        btn = Button(self.root, text="Predict", bg="#95b8ba", font=("goudy old style", 16, "bold"), command=self.predict).place(x=400, y=120)
        
        footer = Label(self.root, text="Music Genre Classification | Machine Learning Project", font=("goudy old style", 16, "bold"), bg="black", fg="white").pack(fill=X, side=BOTTOM)
        
    def upload(self):
        f_types = [('Wav Files', '*.wav')]
        self.file_name = filedialog.askopenfilename(filetypes=f_types)              
        print(self.file_name)
        file = Label(self.root, text=self.file_name, font=("goudy old style", 14, "bold")).place(x=80, y=180) 
        
    def predict(self):
        (rate, sig) = wav.read(self.file_name)            
        mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix, covariance, self.i)
        results = defaultdict(int)                     

        directory = "./Data/genres_original/"

        i = 1
        for folder in os.listdir(directory):
            results[i] = folder
            i += 1
        pred = nearestClass(getNeighbors(dataset, feature, 5))
        print(results[pred])
        resull_label = Label(self.root, text=f"Result : {results[pred]}", bd=2, relief=RIDGE, font=("goudy old style", 24, "bold")).place(x=200, y=260)   
if __name__=='__main__':
    root = Tk()
    obj = App(root, i)
    root.mainloop()
    

C:/Users/win-10/Desktop/Music Genre Classifier/Test/test.wav


classical
